# NO2 and economic activity model

## Addis-Ababa Random Forest + SHAP
Here we load **all 730** daily meshes for Addis, build lag & neighbor features, train a global RF, and visualize SHAP.


#### Imports, constants & helpers

In [16]:
import sys
from pathlib import Path

import pandas as pd
import geopandas as gpd
import numpy as np
import shap

from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# bring src/ into path
CURR_PATH = Path().resolve()
REPO_PATH = CURR_PATH.parent
sys.path.append(str(REPO_PATH / "src"))

# our feature-engineering helpers
from feature_engineering import load_mesh_series, make_lag_features, NeighborAggregator
from feature_engineering import (
    train_rf_pipeline,
    evaluate_model,
    explain_shap,
    plot_shap_dependence,
    compute_elasticities_shap
)

# define the extra numeric features you want:
FEATURE_COLS = [
    "pop_sum_m", "NTL_mean", "road_len", "poi_count",
    "lu_industrial_area", "lu_commercial_area",
    "lu_residential_area","lu_retail_area",
    "lu_farmland_area",  "lu_farmyard_area",
]

# Constants
ADDIS_FOLDER = Path(
    r"C:\Users\Luis.ParraMorales\AirPollution_Analysis"
    r"\air-pollution-mobility-research-project\data"
    r"\Populated meshes\addis-mesh-data"
)
NLAGS   = 7     # cut in half for speed, can tune
K_NEIGH = 8     # number of neighbours

#### Load & build lag features

In [17]:
# load all daily meshes
gdf = load_mesh_series(ADDIS_FOLDER, features=FEATURE_COLS)
print("Total rows:", len(gdf))

C:\Users\Luis.ParraMorales\AirPollution_Analysis\air-pollution-mobility-research-project\src\feature_engineering.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(records, ignore_index=True)


Total rows: 399126


### Create autoregressive lags 1…14 days


In [18]:
df = make_lag_features(gdf, nlags=NLAGS)
df = df.dropna(subset=[f"no2_mean_lag{i}" for i in range(1, NLAGS+1)])
print("Shape after lags:", df.shape)

Shape after lags: (341292, 21)


#### Vectorised neighbour aggregation

In [19]:
# static geometry (center points)
static = gdf.drop_duplicates(["geom_id"])[["geom_id","geometry"]]

# fit neighbour index
neighborer = NeighborAggregator(k=K_NEIGH, id_col="geom_id")
neighborer.fit(static.reset_index(), None)

# 1) unique (geom_id, date) pairs
df_center = df[["geom_id","date"]].drop_duplicates()

# 2) build edge list (center → neighbour)
edges = pd.DataFrame({
    "geom_id": np.repeat(neighborer.ids_, K_NEIGH),
    "neigh_id": neighborer.ids_[neighborer.neigh_idx.ravel()]
})

# 3) cross-join to assign dates to each edge
edges_date = df_center.merge(edges, on="geom_id")

# 4) join lag columns for each neighbour
lag_cols = [f"no2_mean_lag{i}" for i in range(1, NLAGS+1)]
df_lags  = df[["geom_id","date"] + lag_cols]

# this merge will create geom_id_x (center) and geom_id_y (neigh)
df_nei   = edges_date.merge(
    df_lags,
    left_on=["neigh_id","date"],
    right_on=["geom_id","date"],
    how="left"
)

# 5) rename & drop so we group by the *center* geom_id
df_nei = (
    df_nei
    .rename(columns={"geom_id_x": "geom_id"})   # center id
    .drop(columns=["geom_id_y", "neigh_id"])    # drop the neighbour id & duplicate
)

# 6) aggregate the neighbour lags
neigh_feats = (
    df_nei
    .groupby(["geom_id","date"])[lag_cols]
    .mean()
    .rename(columns=lambda c: f"neigh_{c}")
    .reset_index()
)

# 7) merge back onto the full df
df_full = df.merge(neigh_feats, on=["geom_id","date"], how="left")
print("Shape with neighbour feats:", df_full.shape)


Shape with neighbour feats: (341292, 28)


#### Train/test split

In [20]:
# ─── 1) DEFINE RANDOM-SAMPLING TEST SPLIT ─────────────────────────────────────
# all unique dates in the second year
all_2024 = (
    df_full.loc[df_full["date"].dt.year == 2024, "date"]
           .dt.normalize()
           .unique()
)
# sample 10% for test
rng = np.random.default_rng(42)
n_test = max(1, int(len(all_2024) * 0.10))
test_dates = rng.choice(all_2024, size=n_test, replace=False)

# tag rows
df_full["is_test"] = df_full["date"].isin(test_dates)

# build actual splits
train = (
    df_full
    .loc[~df_full["is_test"]]
    .dropna(subset=["no2_mean"])
)
test  = (
    df_full
    .loc[ df_full["is_test"]]
    .dropna(subset=["no2_mean"])
)

print(f"Train days: {len(df_full.loc[~df_full['is_test'], 'date'].dt.normalize().unique())}")
print(f" Test days: {len(test_dates)}")
print(f" Rows → train: {len(train)}, test: {len(test)}")

Train days: 598
 Test days: 30
 Rows → train: 318906, test: 15834


In [21]:
drop_cols = ["no2_mean","geometry","date","is_test"]
X_cols    = [c for c in train.columns if c not in drop_cols]
X_train, y_train = train[X_cols], train["no2_mean"]
X_test,  y_test  = test [X_cols],  test ["no2_mean"]

### Random Forest Pipeline

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# 2) define pipeline (no hard hyperparams here)
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

rf_pipe = Pipeline([
    #("scaler", StandardScaler()),
    ("rf", RandomForestRegressor(
        n_jobs=-1,
        random_state=42,
        oob_score=True
    ))
])

# 3) lean search space
param_dist = {
    "rf__n_estimators":      [100, 150],
    "rf__max_depth":         [10, 15],
    "rf__min_samples_leaf":  [1, 2, 4],
    "rf__max_features":      ["sqrt", "log2"],
    "rf__min_samples_split": [2, 5, 10],      
}

search = RandomizedSearchCV(
    rf_pipe,
    param_distributions=param_dist,
    n_iter=20,
    cv=3,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1,
    random_state=42,
    verbose=1
)

search.fit(X_train, y_train)

best_model = search.best_estimator_
print("🔍 Best hyper-parameters:", search.best_params_)

# 4) evaluate
metrics = evaluate_model(best_model, X_test, y_test)
print("Test RMSE:", metrics["test_rmse"])
print("Test R²:  ", metrics["test_r2"])

import gc, joblib
# free up ~2–4 GB kept by the 60 fitted estimators inside the CV object
del search
joblib.parallel.get_active_backend().abort_everything()  # safety: kill joblib workers
gc.collect()

Fitting 3 folds for each of 20 candidates, totalling 60 fits
🔍 Best hyper-parameters: {'rf__n_estimators': 150, 'rf__min_samples_split': 2, 'rf__min_samples_leaf': 1, 'rf__max_features': 'sqrt', 'rf__max_depth': 20}
Test RMSE: 1.585629573518724e-05
Test R²:   0.37039859808189457


: 

### Fast SHAP on a sub-sample

In [ ]:
X_bg = X_train.sample(min(1000, len(X_train)), random_state=0)
# evaluate SHAP on your RANDOM-SAMPLED test days
explainer, shap_exp = explain_shap(best_model, X_bg, X_test, max_display=15)

### Approximate elasticities from SHAP


In [ ]:
# build full feature matrix (drop objs)
full_X = df_full.dropna(subset=["no2_mean"])[X_cols]

# get a fresh explainer if you like, or re-use the one above
_, full_shap = explain_shap(best_model, X_bg, full_X, max_display=0)  
# max_display=0 suppresses extra plots

elas_df = compute_elasticities_shap(best_model, full_shap)
display(elas_df)